In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

from sklearn.preprocessing import LabelEncoder
import lightgbm as lgbm

In [2]:
import platform                
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False

if platform.system() == 'Darwin': #맥os 사용자의 경우에
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic')
    
elif platform.system() == 'Windows':#윈도우 사용자의 경우에
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.style.use('seaborn-darkgrid') # https://python-graph-gallery.com/199-matplotlib-style-sheets/
    rc('font', family=font_name)

In [278]:
df = pd.read_csv('real_final.csv', index_col = 0)
df.head()

,방송일시,노출(분),상품군,판매단가,취급액,결제유형,월,일,시,분,...,1,2,3,4,5,6,7,8,상품명,취급액_log
0,2019-01-01 06:00:00,60.0,의류,39900,2099000.0,0,1,1,6,0,...,0.0,0.0,0.0,0.0,2.766667,0.0,0.0,0.0,테이트 남성 셀린니트3종,14.556972
1,2019-01-01 06:00:00,60.0,의류,39900,4371000.0,0,1,1,6,0,...,0.0,0.0,0.0,0.0,2.766667,0.0,0.0,0.0,테이트 여성 셀린니트3종,15.290503
2,2019-01-01 06:20:00,60.0,의류,39900,3262000.0,0,1,1,6,20,...,0.0,0.0,0.0,0.0,2.111111,0.0,0.0,0.0,테이트 남성 셀린니트3종,14.997851
3,2019-01-01 06:20:00,60.0,의류,39900,6955000.0,0,1,1,6,20,...,0.0,0.0,0.0,0.0,2.111111,0.0,0.0,0.0,테이트 여성 셀린니트3종,15.754972
4,2019-01-01 06:40:00,60.0,의류,39900,6672000.0,0,1,1,6,40,...,0.0,0.0,0.0,0.0,1.455556,0.0,0.0,0.0,테이트 남성 셀린니트3종,15.713430


In [226]:
sub = pd.read_excel('./2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', header = 1)
sub.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


In [279]:
df.drop(['방송일시', '노출(분)', '취급액', '분', '상품명'], axis = 1, inplace = True)
df.head()

,상품군,판매단가,결제유형,월,일,시,소비층,요일,휴일,프라임,...,0,1,2,3,4,5,6,7,8,취급액_log
0,의류,39900,0,1,1,6,1,Tuesday,2,4,...,13.0,0.0,0.0,0.0,0.0,2.766667,0.0,0.0,0.0,14.556972
1,의류,39900,0,1,1,6,1,Tuesday,2,4,...,13.0,0.0,0.0,0.0,0.0,2.766667,0.0,0.0,0.0,15.290503
2,의류,39900,0,1,1,6,1,Tuesday,2,4,...,13.0,0.0,0.0,0.0,0.0,2.111111,0.0,0.0,0.0,14.997851
3,의류,39900,0,1,1,6,1,Tuesday,2,4,...,13.0,0.0,0.0,0.0,0.0,2.111111,0.0,0.0,0.0,15.754972
4,의류,39900,0,1,1,6,1,Tuesday,2,4,...,13.0,0.0,0.0,0.0,0.0,1.455556,0.0,0.0,0.0,15.713430


In [280]:
df.columns

Index(['상품군', '판매단가', '결제유형', '월', '일', '시', '소비층', '요일', '휴일', '프라임', '순서',
       '마더코드', '상품코드', '노출시간(각)', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '취급액_log'],
      dtype='object')

## Finding Embedding Vector 

In [281]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state = 123457)

In [282]:
train_y = train.pop('취급액_log')
train_x = train
test_y = test.pop('취급액_log')
test_x = test

In [284]:
categorical_cols = ['상품군','결제유형', '월', '일', '시', '소비층', '요일', '휴일', '프라임', '순서', '마더코드', '상품코드']
other_cols = [i for i in train_x.columns if i not in categorical_cols]
other_cols = other_cols[:2]
print(len(categorical_cols), len(other_cols))

12 2


In [285]:
#Categorical Preprocessing
def preproc(df):
    input_list_train = []
    for c in categorical_cols:
        names = np.asarray(df[c].tolist())
        names = pd.factorize(names)[0]
        input_list_train.append(np.asarray(names))
    input_list_train.append(df[other_cols].values)
    return input_list_train

In [286]:
df_tr = preproc(train_x)
df_val = preproc(test_x)

In [288]:
#Translation
categorical_cols_eng = ['item_cat', 'pat_type', 'month', 'date', 'hour', 'consumer_type', 'day', 'holday',
                        'prime', 'order','mothercode', 'itemcode']

In [289]:
#Input Categorical columns
ins = []
models = []
for i, cat in enumerate(categorical_cols):
    var = df[cat].nunique()
    embedding_size = 1
    model = Input((1,), name = str(i) + '_model')
    ins.append(model)
    model = Embedding(var, embedding_size, input_length = 1, name = categorical_cols_eng[i])(model)
    model = Reshape(target_shape = (embedding_size,))(model)
    models.append(model)

In [290]:
#Continuous columns
cont = Input((len(other_cols),), name = 'cont_vars')
ins.append(cont)
models.append(cont)

In [292]:
K.clear_session()

embed_model = Concatenate()(models)
embed_model = Dense(50, activation = 'relu')(embed_model)
embed_model = Dropout(0.4)(embed_model)
embed_model = Dense(100, activation = 'relu')(embed_model)
embed_model = Dropout(0.4)(embed_model)
embed_model = Dense(50, activation = 'relu')(embed_model)
embed_model = Dropout(0.4)(embed_model)
embed_model = Dense(1)(embed_model)

final_model = Model(inputs = ins, outputs = embed_model)
final_model.compile(loss = 'mse', optimizer = 'Adam',
                        metrics = ['mse'])

In [293]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
0_model (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
1_model (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
2_model (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
3_model (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [294]:
callback = EarlyStopping(monitor='loss', patience=10)

In [295]:
history = final_model.fit(df_tr, train_y, epochs = 100, batch_size = 64,
                          validation_data = [df_val, test_y],verbose = 1, callbacks = [callback])

Train on 28300 samples, validate on 7075 samples
Epoch 1/100
28300/28300 [==============================] - 3s 96us/sample - loss: 685289024.2411 - mse: 685288832.0000 - val_loss: 312710.7934 - val_mse: 312710.7812
Epoch 2/100
28300/28300 [==============================] - 1s 41us/sample - loss: 32342448.3233 - mse: 32342444.0000 - val_loss: 38349.3006 - val_mse: 38349.3008
Epoch 3/100
28300/28300 [==============================] - 1s 40us/sample - loss: 9786861.7296 - mse: 9786861.0000 - val_loss: 399.4043 - val_mse: 399.4042
Epoch 4/100
28300/28300 [==============================] - 1s 43us/sample - loss: 4303694.8382 - mse: 4303693.5000 - val_loss: 726.2803 - val_mse: 726.2804
Epoch 5/100
28300/28300 [==============================] - 1s 42us/sample - loss: 2426302.1312 - mse: 2426303.0000 - val_loss: 492.5355 - val_mse: 492.5356
Epoch 6/100
28300/28300 [==============================] - 1s 44us/sample - loss: 1292564.9554 - mse: 1292564.8750 - val_loss: 379.9523 - val_mse: 379.9524

Epoch 55/100
28300/28300 [==============================] - 1s 44us/sample - loss: 18.3239 - mse: 18.3239 - val_loss: 15.4189 - val_mse: 15.4189
Epoch 56/100
28300/28300 [==============================] - 1s 43us/sample - loss: 104.7370 - mse: 104.7369 - val_loss: 12.9181 - val_mse: 12.9181
Epoch 57/100
28300/28300 [==============================] - 1s 43us/sample - loss: 11.8311 - mse: 11.8311 - val_loss: 10.6463 - val_mse: 10.6463
Epoch 58/100
28300/28300 [==============================] - 1s 43us/sample - loss: 20.1988 - mse: 20.1988 - val_loss: 8.6459 - val_mse: 8.6459
Epoch 59/100
28300/28300 [==============================] - 1s 43us/sample - loss: 53.5739 - mse: 53.5739 - val_loss: 6.9611 - val_mse: 6.9611
Epoch 60/100
28300/28300 [==============================] - 1s 43us/sample - loss: 11.6210 - mse: 11.6210 - val_loss: 5.5146 - val_mse: 5.5146
Epoch 61/100
28300/28300 [==============================] - 1s 43us/sample - loss: 4.9742 - mse: 4.9742 - val_loss: 4.3019 - val_mse: 

In [296]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [297]:
mean_absolute_percentage_error(np.expm1(test_y.values), np.exp(final_model.predict(df_val).reshape(-1, 1)))

135.38553016749918

In [298]:
itemcode_list = []
for i in final_model.get_layer('itemcode').get_weights()[0]:
    itemcode_list.append(i[0])
itemcode_list = np.array(itemcode_list)

In [299]:
mothercode_list = []
for i in final_model.get_layer('mothercode').get_weights()[0]:
    mothercode_list.append(i[0])
mothercode_list = np.array(mothercode_list)


In [300]:
dict = {'itmecode': df['상품코드'].unique(), 'embed': itemcode_list}
itemcode_df = pd.DataFrame(dict)

In [301]:
dict = {'mothercode': df['마더코드'].unique(), 'embed': mothercode_list}
mothercode_df = pd.DataFrame(dict)

In [302]:
df = pd.merge(df, embed_df, left_on = '상품코드', right_on = 'itemcode').drop('itemcode', axis = 1)

,상품군,판매단가,결제유형,월,일,시,소비층,요일,휴일,프라임,...,1,2,3,4,5,6,7,8,취급액_log,embed
0,의류,39900,0,1,1,6,1,Tuesday,2,4,...,0.0,0.000000,0.000000,0.0,2.766667,0.0,0.000000,0.0,14.556972,-0.014405
1,의류,39900,0,1,1,6,1,Tuesday,2,4,...,0.0,0.000000,0.000000,0.0,2.111111,0.0,0.000000,0.0,14.997851,-0.014405
2,의류,39900,0,1,1,6,1,Tuesday,2,4,...,0.0,0.000000,0.000000,0.0,1.455556,0.0,0.000000,0.0,15.713430,-0.014405
3,의류,39900,0,1,3,0,1,Thursday,0,4,...,0.0,0.477143,0.040000,0.0,2.000000,0.0,0.026575,0.0,15.807350,-0.014405
4,의류,39900,0,1,3,0,1,Thursday,0,4,...,0.0,0.476190,0.040667,0.0,2.000000,0.0,0.027067,0.0,16.165075,-0.014405


In [303]:
df = pd.merge(df, mothercode_df, left_on = '마더코드', right_on = 'mothercode').drop('mothercode', axis = 1)

In [304]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35375 entries, 0 to 35374
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   상품군      35375 non-null  object 
 1   판매단가     35375 non-null  int64  
 2   결제유형     35375 non-null  int64  
 3   월        35375 non-null  int64  
 4   일        35375 non-null  int64  
 5   시        35375 non-null  int64  
 6   소비층      35375 non-null  int64  
 7   요일       35375 non-null  object 
 8   휴일       35375 non-null  int64  
 9   프라임      35375 non-null  int64  
 10  순서       35375 non-null  int64  
 11  마더코드     35375 non-null  int64  
 12  상품코드     35375 non-null  int64  
 13  노출시간(각)  35375 non-null  float64
 14  0        35375 non-null  float64
 15  1        35375 non-null  float64
 16  2        35375 non-null  float64
 17  3        35375 non-null  float64
 18  4        35375 non-null  float64
 19  5        35375 non-null  float64
 20  6        35375 non-null  float64
 21  7        353

In [305]:
df['결제유형'] = df['결제유형'].astype('str')
df['소비층'] = df['소비층'].astype('str')
df['휴일']= df['휴일'].astype('str')
df['프라임'] = df['프라임'].astype('str')
df.drop(['마더코드', '상품코드', '0', '1', '2', '3', '4', '5', '6', '7', '8'], axis = 1, inplace = True)

In [306]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [307]:
#Transalting Columns
df_num.columns = ['item_category', 'unit_price', 'pay_type', 'month', 'date', 'time', 'consumer_type',
                 'day', 'holiday', 'prime', 'order', 'exposure_time', 'pay_amount(log)', 'embed_x', 'embed_y']

In [308]:
train, val = train_test_split(df_num, test_size = 0.2, random_state = 123457)

In [309]:
x_train = train.drop(['pay_amount(log)', 'embed_x', 'embed_y'], axis = 1)
y_train = train['pay_amount(log)']
x_val = val.drop(['pay_amount(log)', 'embed_x', 'embed_y'], axis = 1)
y_val = val['pay_amount(log)']

In [310]:
train_ds = lgbm.Dataset(x_train, label=y_train)
val_ds = lgbm.Dataset(x_val, label=y_val)

params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'mape',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

model = lgbm.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 10
                 )

Training until validation scores don't improve for 10 rounds
[100]	valid_0's mape: 0.028897
[200]	valid_0's mape: 0.0265063
[300]	valid_0's mape: 0.0254625
[400]	valid_0's mape: 0.0247521
[500]	valid_0's mape: 0.0241371
[600]	valid_0's mape: 0.0236747
[700]	valid_0's mape: 0.023349
[800]	valid_0's mape: 0.0231178
[900]	valid_0's mape: 0.0229283
[1000]	valid_0's mape: 0.0227585
Did not meet early stopping. Best iteration is:
[1000]	valid_0's mape: 0.0227585


In [311]:
model.predict(x_val)

array([14.96459418, 16.0540099 , 16.59263769, ..., 16.86789562,
       15.4991743 , 16.21926163])

In [312]:
mean_absolute_percentage_error(np.expm1(y_val), np.exp(model.predict(x_val)))

45.042398507809